In [1]:
import numpy as np
import pandas as pd

In [2]:
# 6.2 이진 데이터 형식

In [3]:
# 데이터를 효율적으로 저장하는 가장 손쉬운 방법은 파이썬에 기본으로 내장되어 있는 pickle 직렬화를 사용해 데이터를 이진 형식으로 저장하는 것이다. 

In [4]:
# 편리하게도 pandas 객체는 모두 pickle을 이용해서 데이터를 저장하는 to_pickle 메서드를 가지고 있다.

In [5]:
frame = pd.read_csv("examples/ex1.csv")

In [6]:
frame

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [7]:
frame.to_pickle("examples/frame_pickle")

In [8]:
# pickle로 직렬화된 객체는 내장 함수인 pickle로 직접 불러오거나 아니면 좀 더 편리한 pickle 함수인 pandas.read_pickle 메서드를 이용하여 불러올 수 있다.

In [9]:
pd.read_pickle("examples/frame_pickle")

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [10]:
# 주의! pickle은 오래 보관할 필요가 없는 데이터일 경우에만 추천한다. 오랜 시간이 지나도 안정적으로 데이터를 저장할 거라고 보장하기 힘든 문제가 있기 때문이다.

In [11]:
# pandas는 HDF5와 Message-Pack, 두 가지 바이너리 포맷을 지원한다. 

In [12]:
# 다음 절에서 HDF5 예제를 살펴보겠지만 다양한 파일 형식이 실제 독자들의 분석 작업에 얼마나 더 적절한지 직접 살펴봐야할 필요가 있다.

In [13]:
# 다음과 같은 pandas 또는 NumPy 데이터를 위한 다른 저장 형식도 존재한다.

In [14]:
# Bcolz : Blocs 압축 알고리즘에 기반한 압축이 가능한 컬럼지향 바이너리 포맷이다.

In [15]:
# Feather : R 커뮤니티의 해들리 위컴과 저자가 함께 설계한 컬럼지향 파일 형식이다. Feather는 아파치 에로우의 메모리 포맷을 사용한다.

In [16]:
# 6.2.1 HDF5 형식 사용하기

In [17]:
# HDF5는 대량의 과학 계산용 배열 데이터를 저장하기 위해 고안된 훌륭한 파일 포맷이다.

In [18]:
# C 라이브러리로도 존재하며 자바, 줄리아, 매트랩, 그리고 파이썬 같은 다양한 다른 언어에서도 사용할 수 있는 인터페이스를 제공한다.

In [19]:
# HDF는 Hiearchical Data Format의 약자로 계층적 데이터 형식이라는 뜻이다. 각각의 HDF5 파일은 여러 개의 데이터셋을 저장하고 부가 정보를 기록할 수 있다.

In [20]:
# 보다 단순한 형식과 비교하면 HDF5는 다양한 압축 기술을 사용해서 실시간 압축을 지원하며 반복되는 패턴을 가진 데이터를 좀 더 효과적으로 저장할 수 있다.

In [21]:
# 메모리에 모두 적재할 수 없는 엄청나게 큰 데이터를 아주 큰 배열에서 필요한 작은 부분들만 효과적으로 읽고 쓸 수 있는 훌륭한 선택이다.

In [22]:
# PyTables나 h5py 라이브러리를 이용해서 직접 HDF5 파일에 접근하는 것도 가능하지만 pandas는 Series나 DataFrame 객체로 간단히 저장할 수 있는 고수준의 인터페이스를 제공한다.

In [23]:
# HDFStore 클래스는 사전처럼 작동하며 세밀한 요구 사항도 잘 처리해준다.

In [24]:
frame = pd.DataFrame({"a": np.random.randn(100)})

In [26]:
store = pd.HDFStore("mydata.h5")

In [27]:
store["obj1"] = frame

In [28]:
store["obj1_col"] = frame["a"]

In [29]:
store

<class 'pandas.io.pytables.HDFStore'>
File path: mydata.h5

In [31]:
# HDF5 파일에 포함된 객체는 파이썬 사전과 유사한 형식으로 사용 가능하다.

In [32]:
store["obj1"]

,a
0,-0.132316
1,-2.126733
2,0.435198
3,0.880628
4,-0.420367
...,...
95,1.519283
96,1.517627
97,-0.393811
98,0.693759


In [33]:
# HDFStore는 "fixed"와 "table" 두 가지 저장 스키마를 지원한다.

In [34]:
# "table" 스키마가 일반적으로 더 느리지만 아래와 같은 특별한 문법을 이용해 쿼리 연산을 지원한다.

In [36]:
store.put("obj2", frame, format="table")

In [37]:
store.select("obj2", where=["index >= 10 and index <= 15"])

,a
10,0.439588
11,-1.667062
12,0.215301
13,0.161834
14,-0.594897
15,0.381450


In [38]:
store.close()

In [39]:
# put은 명시적인 store["obj2"] = frame 메서드지만 저장 스키마를 지정하는 등의 다른 옵션을 제공한다. 

In [40]:
# pandas.read_hdf 함수는 이런 기능들을 축약해서 사용할 수 있다.

In [41]:
frame.to_hdf("mydata.h5", "obj3", format="table")

In [42]:
pd.read_hdf("mydata.h5", "obj3", where=["index < 5"])

,a
0,-0.132316
1,-2.126733
2,0.435198
3,0.880628
4,-0.420367


In [43]:
# 만약 로컬 스토리지에서 엄청난 양의 데이터를 다뤄야 한다면 PyTables와 h5py를 살펴보고 목적에 맞는지 알아봐야 한다.

In [44]:
# 실제로 대부분의 데이터 분석 문제는 CPU보다는 IO 성능에 의존적이므로 HDF5 같은 도구를 사용하면 애플리케이션의 성능을 매우 향상시킬 수 있다.